# Roboflow 활용하기(이어서)

이번 챕터에서는 roboflow 서비스를 이용한

커스텀 모델 구축 후, 이 모델을 로컬에서 사용하는

다양한 방법에 대해 보여드리겠습니다.

우선 roboflow에서 제공하는 두 가지 모듈인

roboflow와 inference를 가상환경에 설치합니다.

In [ ]:
%pip install inference roboflow supervision

만약 CUDA지원 가능한 NVIDIA GPU가 설치된 PC라면

inference 대신 inference-gpu를 설치하셔야 GPU 활용 가능합니다.

## 중요 : 환경변수 추가

그리고 roboflow에서 제공받은 api-key를 환경변수에 추가하셔야 합니다.

환경변수 이름은 ROBOFLOW_API_KEY입니다.

내 코드에 환경변수를 그대로 적어놓으면 노출의 위험성이 있습니다.
 

In [1]:
import inference

# 아래 명령을 실행하면 로컬에 모델이 onnx 타입으로 저장됨.
inference.models.yolov8.yolov8_object_detection
model = inference.get_roboflow_model("penguin-detection-qyh4k/1")

# 저장경로는 model.cache_dir로 확인 가능
print(model.cache_dir)

C:\Users\Administrator\PycharmProjects\jss_tutorials\.venv\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


/tmp/cache\penguin-detection-qyh4k/1


In [3]:
# cache 안에 다운받은 onnx 파일이 있으면
# 다시 다운로드하지 않고 해당 파일을 계속 사용합니다.
# 기본적인 추론 방법은 아래와 같습니다.

orig_img = "test/p_on_water.jpg"
results = model.infer(image=orig_img)
pred = results[0].predictions[0]

# 추론결과의 데이터는 아래와 같습니다.
# ultralytics와는 달리,
# 원본 이미지(orig_img) 배열이 포함되어 있지 않습니다. 
print(pred)

x=404.5 y=266.0 width=133.0 height=92.0 confidence=0.8281735181808472 class_name='penguin' class_confidence=None class_id=0 tracker_id=None


추론결과의 클래스이름과 신뢰도, xywh좌표가 나와 있습니다.

이를 통해 ultralytics로 추출할 때와 유사한 방식으로

추론 후의 이미지를 직접 만들어낼 수 있습니다.

In [4]:
import cv2  # ImportError 발생시 pip install opencv-python 실행

def pred_to_xyxy(pred):
    "roboflow의 xywh를 xyxy로 변환하는 함수"
    x1, y1 = pred.x-pred.width/2, pred.y-pred.height/2
    x2, y2 = pred.x+pred.width/2, pred.y+pred.height/2
    return int(x1), int(y1), int(x2), int(y2)

im0 = cv2.imread(orig_img)  # 이미지를 배열로 불러옴
x1, y1, x2, y2 = pred_to_xyxy(pred)

# 직접 웹캠이미지에 네모 그리기
cv2.rectangle(img=im0,  # 파일명이 아니라 배열을 넣어야 함
              pt1=(x1, y1),
              pt2=(x2, y2),
              color=(255, 0, 255),
              thickness=3)

org = [x1, y1]  # 주석 작성할 좌표
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0)
thickness = 2
cv2.putText(im0, pred.class_name + f" {pred.confidence:.02f}", org, font, fontScale, color, thickness)

cv2.imwrite(filename=f"./result.jpg", img=im0)
cv2.imshow('result', mat=im0)  # opencv 창에 이미지 갱신
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
from ultralytics import YOLO

# 커스텀 pt인 경우에는 task를 지정해주어야 작동합니다.
model = YOLO('./model/weights.onnx', task="detect")

In [43]:
model.predict("test/p_on_water.jpg", save=True)

image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\test\p_on_water.jpg: 640x640 1 penguin, 104.7ms
Speed: 4.6ms preprocess, 104.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\detect\predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'penguin'}
 obb: None
 orig_img: array([[[134, 137, 142],
         [ 82,  85,  90],
         [ 74,  76,  84],
         ...,
         [ 85,  84, 104],
         [ 78,  77,  97],
         [ 91,  95, 114]],
 
        [[104, 107, 112],
         [ 69,  72,  77],
         [ 59,  61,  69],
         ...,
         [107, 106, 126],
         [108, 107, 127],
         [ 96, 100, 119]],
 
        [[ 52,  55,  60],
         [ 52,  55,  60],
         [ 53,  55,  63],
         ...,
         [128, 127, 147],
         [113, 112, 132],
         [ 94,  95, 115]],
 
        ...,
 
        [[ 77,  87, 111],
         [103, 113, 137],
         [105, 115, 139],
         ...,
         [ 96, 105, 125],
         [ 99, 108, 128],
         [ 82,  91, 111]],
 
        [[ 99, 109, 133],
         [ 93, 103, 127],
         [ 93, 103, 127],
         ...,
         [ 95, 10

# 편하긴 한데, 더 저렴한 방법이 없나?

지금까지, roboflow에서 학습을 완료하고

다운받은 onnx 모델을 사용해서 추론해보았습니다.

하지만, 위의 방법은 꽤 비용이 든다고 말씀드렸습니다.

> (249$/mon 을 기억하시기 바랍니다.)

위와 같이 roboflow에서 제공하는 

(비싸지만) 편리한 기능들을 사용하지 않고

roboflow에서 레이블링만 완료하고

데이터셋을 로컬 또는 코랩으로 다운받아 

직접 학습하여 pt 파일을 만드는 방법입니다.

(코랩도 돈이 듭니다.)


In [31]:
from roboflow import Roboflow

rf = Roboflow("un7kANfGUhngOf2sYCk3")
# proj = rf.workspace("Martin Shin")
# proj = rf.workspace("penguin-detection-qyh4k/1")
prj = rf.workspace("martin-shin-ipe9b").project('penguin-detection-qyh4k')
dataset = prj.version(1).download('yolov8')

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.6, to fix: `pip install ultralytics==8.0.196`
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to penguin-detection-1 in yolov8:: 100%|██████████| 108/108 [00:00<00:00, 2454.70it/s]


In [34]:
# dataset.location을 열어보면 
# 데이터셋 다운로드 경로가 나옵니다.

dataset.location

'C:\\Users\\Administrator\\PycharmProjects\\jss_tutorials\\6_custom_dataset\\penguin-detection-1'

다운로드한 데이터셋의 data.yaml을 열어보면 

아래와 같습니다.

- penguin
nc: 1
roboflow:
  license: CC BY 4.0
  project: penguin-detection-qyh4k
  url: https://universe.roboflow.com/martin-shin-ipe9b/penguin-detection-qyh4k/dataset/1
  version: 1
  workspace: martin-shin-ipe9b
test: test/images
train: train/images
val: valid/images

가끔 데이터셋의 경로가 잘못 기재되는 경우가 있는데,

test, train, val을 적절히 설정해주면 됩니다.

예를 들어 제 경우에는 기존의 아래 세 라인을

test: ../test/images
train: penguin-detection-1/train/images
val: penguin-detection-1/valid/images

아래와 같이 수정하였습니다.

test: test/images
train: train/images
val: val/images


아래부터는 로컬에서 학습하는 코드입니다.

(시간관계상 10에포크만 수행해보았습니다.)

제 랩탑 기준으로 약 8분 정도 걸리네요.

일반적으로 100에포크 전후로 돌리기는 합니다.


In [35]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640

New https://pypi.org/project/ultralytics/8.1.9 available �윑� Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.6 �윓� Python-3.10.11 torch-2.1.2+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\penguin-detection-1/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment


train: Scanning C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\penguin-detection-1\train\labels...:   0%|          | 0/42 [00:00<?, ?it/s]
train: Scanning C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\penguin-detection-1\train\labels... 42 images, 9 backgrounds, 0 corrupt: 100%|##########| 42/42 [00:00<00:00, 1678.38it/s]

val: Scanning C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\penguin-detection-1\valid\labels...:   0%|          | 0/4 [00:00<?, ?it/s]
val: Scanning C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\penguin-detection-1\valid\labels... 4 images, 1 backgrounds, 0 corrupt: 100%|##########| 4/4 [00:00<00:00, 1283.35it/s]

  0%|          | 0/3 [00:00<?, ?it/s]
       1/10         0G      1.363      3.739      1.398         43        640:   0%|          | 0/3 [00:10<?, ?it/s]
       1/10         0G      1.363      3.739      1.398         43        640:  33%|###3      | 1/3 [00:10<0


학습이 완료되었습니다.

메시지의 `Results saved to ~` 에 있는 경로로 찾아가서

best.pt 파일을 현재 폴더로 복사한 후에

이전과 동일하게 아래 방식으로 코드를 실행하시면 됩니다.


In [39]:
model = YOLO('best.pt')

In [40]:
model.predict(orig_img, save=True)

image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\6_custom_dataset\test\p_on_water.jpg: 480x640 1 penguin, 193.7ms
Speed: 3.3ms preprocess, 193.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\detect\predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'penguin'}
 obb: None
 orig_img: array([[[134, 137, 142],
         [ 82,  85,  90],
         [ 74,  76,  84],
         ...,
         [ 85,  84, 104],
         [ 78,  77,  97],
         [ 91,  95, 114]],
 
        [[104, 107, 112],
         [ 69,  72,  77],
         [ 59,  61,  69],
         ...,
         [107, 106, 126],
         [108, 107, 127],
         [ 96, 100, 119]],
 
        [[ 52,  55,  60],
         [ 52,  55,  60],
         [ 53,  55,  63],
         ...,
         [128, 127, 147],
         [113, 112, 132],
         [ 94,  95, 115]],
 
        ...,
 
        [[ 77,  87, 111],
         [103, 113, 137],
         [105, 115, 139],
         ...,
         [ 96, 105, 125],
         [ 99, 108, 128],
         [ 82,  91, 111]],
 
        [[ 99, 109, 133],
         [ 93, 103, 127],
         [ 93, 103, 127],
         ...,
         [ 95, 10

아래와 같은 이미지가 만들어졌네요.

![](https://i.ibb.co/MNJcnR1/p-on-water.jpg)